In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import collections
import pickle
import numpy as np

In [51]:
test = pd.read_csv("test1_combined.csv")
df = pd.read_csv("train1_combined.csv")
dev = pd.read_csv("dev1_combined.csv")
train_full = pd.read_csv('train_transformed9feat.csv')
test_full = pd.read_csv('test_transformed9feat.csv')
with open('training_labels_combined.ob', 'rb') as f:
    train_full_label = pickle.load(f)

#test.drop(test.columns[[13,14]], axis=1, inplace=True)
#df.drop(df.columns[[13,14]], axis=1, inplace=True)

In [104]:
train_full_label = train_full_label.tolist()

In [105]:
train_full = train_full.iloc[:,4:]
print(train_full)

             aa        ra        jc   pa  path_len      dc_A      dc_B  \
0      2.471649  0.531746  0.428571   96         1  0.002124  0.003186   
1      2.976054  0.668651  0.466667  112         1  0.002124  0.003717   
2      2.899858  0.628968  0.700000   72         1  0.002124  0.002390   
3      2.471649  0.531746  0.600000   64         1  0.002124  0.002124   
4      2.976054  0.668651  0.466667  112         1  0.002124  0.003717   
...         ...       ...       ...  ...       ...       ...       ...   
32067  0.000000  0.000000  0.000000  510         3  0.002655  0.001328   
32068  0.000000  0.000000  0.000000   30       999  0.002655  0.001593   
32069  0.000000  0.000000  0.000000  310         3  0.002655  0.002390   
32070  0.000000  0.000000  0.000000  140         3  0.002655  0.001062   
32071  0.000000  0.000000  0.000000   60         4  0.002655  0.001062   

         bet_cA        bet_cB  
0      0.000004  7.627580e-04  
1      0.000004  7.945610e-04  
2      0.000004

In [106]:
test_full = test_full.iloc[:,4:]

In [107]:
test_full.shape

(2000, 9)

In [108]:
train_full.shape

(32072, 9)

In [39]:
from sklearn.utils import shuffle
df_shuffled = shuffle(df, random_state=785291)
df_shuffled.reset_index(drop=True)

,Unnamed: 0,Author_A,Author_B,count,aa,ra,jc,pa,dc_A,dc_B,sh_path,label
0,8763,3005,3863,1,1.213667,0.268868,0.176471,64,0.004635,0.001159,1,1
1,11885,629,1947,1,0.000000,0.000000,0.000000,14,0.000579,0.006373,5,0
2,2339,1518,915,1,0.000000,0.000000,0.000000,21,0.000869,0.002607,4,0
3,4059,1937,2161,1,0.000000,0.000000,0.000000,256,0.008111,0.001738,6,0
4,11905,657,1802,4,7.113244,1.121676,0.733333,675,0.007242,0.007822,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
27353,3606,183,3716,1,0.000000,0.000000,0.000000,36,0.001738,0.002897,4,0
27354,8480,3084,1816,1,0.000000,0.000000,0.000000,10,0.000579,0.000579,4,0
27355,92,1017,1784,1,0.000000,0.000000,0.000000,24,0.000579,0.000579,999,0
27356,7313,2559,3808,3,0.434294,0.100000,0.100000,18,0.002607,0.000579,1,1


In [40]:
df_shuffled.iloc[:,4:-1]

,aa,ra,jc,pa,dc_A,dc_B,sh_path
8763,1.213667,0.268868,0.176471,64,0.004635,0.001159,1
25564,0.000000,0.000000,0.000000,14,0.000579,0.006373,5
16018,0.000000,0.000000,0.000000,21,0.000869,0.002607,4
17738,0.000000,0.000000,0.000000,256,0.008111,0.001738,6
11905,7.113244,1.121676,0.733333,675,0.007242,0.007822,1
...,...,...,...,...,...,...,...
17285,0.000000,0.000000,0.000000,36,0.001738,0.002897,4
22159,0.000000,0.000000,0.000000,10,0.000579,0.000579,4
13771,0.000000,0.000000,0.000000,24,0.000579,0.000579,999
7313,0.434294,0.100000,0.100000,18,0.002607,0.000579,1


In [52]:
x_train_all = df.iloc[:,4:-1]
y_train_all = df.label
x_dev = dev.iloc[:,4:-1]
y_dev = dev.label

In [68]:
x_train_all

,aa,ra,jc,pa,dc_A,dc_B,sh_path
0,2.508376,0.551129,0.461538,88,0.002317,0.003187,1
1,3.049436,0.707973,0.500000,104,0.002317,0.003766,1
2,2.984187,0.673785,0.700000,72,0.002317,0.002607,1
3,2.027478,0.426129,0.500000,56,0.002317,0.002028,1
4,3.060384,0.713467,0.466667,112,0.002317,0.004056,1
...,...,...,...,...,...,...,...
27353,0.000000,0.000000,0.000000,51,0.002607,0.000290,3
27354,0.000000,0.000000,0.000000,27,0.002607,0.000290,999
27355,0.000000,0.000000,0.000000,207,0.002607,0.002317,3
27356,0.000000,0.000000,0.000000,27,0.002607,0.003476,3


In [97]:
splits = [0.2, 0.4, 0.6, 0.8]
x_train2 = []
y_train2 = []
for i in range(len(splits)):
    x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, train_size=splits[i], random_state=42, stratify=y_train_all)
    x_train2.append(x_test)
    y_train2.append(y_test)
x_train2.insert(0,x_train_all)
y_train2.insert(0,y_train_all)

In [80]:
x_train[0].shape

(21886,)

In [14]:
x_dev.shape

(10110, 7)

In [6]:
len(dev[dev.label ==0])

5055

In [19]:
df.iloc[:,4:-1]

,aa,ra,jc,pa,dc_A,dc_B,sh_path
0,2.508376,0.551129,0.461538,88,0.002317,0.003187,1
1,3.049436,0.707973,0.500000,104,0.002317,0.003766,1
2,2.984187,0.673785,0.700000,72,0.002317,0.002607,1
3,2.027478,0.426129,0.500000,56,0.002317,0.002028,1
4,3.060384,0.713467,0.466667,112,0.002317,0.004056,1
...,...,...,...,...,...,...,...
27353,0.000000,0.000000,0.000000,51,0.002607,0.000290,3
27354,0.000000,0.000000,0.000000,27,0.002607,0.000290,999
27355,0.000000,0.000000,0.000000,207,0.002607,0.002317,3
27356,0.000000,0.000000,0.000000,27,0.002607,0.003476,3


In [20]:
test.iloc[:,4:]

,aa,ra,jc,pa,dc_A,dc_B,sh_path
0,0.000000,0.000000,0.00,40,0.002317,0.001448,3
1,0.000000,0.000000,0.00,24,0.002317,0.000869,6
2,0.000000,0.000000,0.00,432,0.004635,0.007822,4
3,1.242670,0.400000,0.10,57,0.005504,0.000869,1
4,0.000000,0.000000,0.00,391,0.004925,0.006663,4
...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.00,2,0.000290,0.000579,1
1996,0.000000,0.000000,0.00,2,0.000579,0.000290,999
1997,0.000000,0.000000,0.00,42,0.000579,0.006083,3
1998,1.508374,0.404053,0.05,236,0.017092,0.001159,1


In [78]:
y = [True if i < df.shape[0]/2 else False for i in range(df.shape[0])]
x = df.iloc[:,4:13]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [191]:
collections.Counter(y)

Counter({True: 13679, False: 13679})

## Logistic Regression full training set

In [109]:
LR = LogisticRegression(n_jobs = 4, penalty = 'l1', solver = 'liblinear').fit(train_full, train_full_label)
collections.Counter(LR.predict(test_full))


C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


Counter({0.0: 1245, 1.0: 755})

In [110]:
results = LR.predict_proba(test_full)
results[:,1]

array([8.54274468e-04, 3.57277437e-09, 6.01940118e-04, ...,
       8.14453865e-04, 9.99999127e-01, 3.59846951e-09])

In [111]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg_7feat_hyper_full.csv', header = True, index= False)

## Logistic Regression development on dev set

In [99]:
x_train2[0]

,aa,ra,jc,pa,dc_A,dc_B,sh_path
0,2.508376,0.551129,0.461538,88,0.002317,0.003187,1
1,3.049436,0.707973,0.500000,104,0.002317,0.003766,1
2,2.984187,0.673785,0.700000,72,0.002317,0.002607,1
3,2.027478,0.426129,0.500000,56,0.002317,0.002028,1
4,3.060384,0.713467,0.466667,112,0.002317,0.004056,1
...,...,...,...,...,...,...,...
27353,0.000000,0.000000,0.000000,51,0.002607,0.000290,3
27354,0.000000,0.000000,0.000000,27,0.002607,0.000290,999
27355,0.000000,0.000000,0.000000,207,0.002607,0.002317,3
27356,0.000000,0.000000,0.000000,27,0.002607,0.003476,3


In [104]:
#from sklearn.preprocessing import StandardScaler
#normal = StandardScaler()
#x_train = normal.fit_transform(x_train)
#x_test = normal.transform(x_test)
#l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'
auc_train = []
auc_dev = []
for i in range(len(x_train2)):
    LR = LogisticRegression(penalty = 'l1', solver = 'liblinear').fit(x_train2[i], y_train2[i])
    probs_train = LR.predict_proba(x_train_all)
    probs_dev = LR.predict_proba(dev.iloc[:,4:-1])
    auc_train.append(metrics.roc_auc_score(y_train_all, probs_train[:, 1]))
    auc_dev.append(metrics.roc_auc_score(dev.label, probs_dev[:, 1]))

#print(metrics.classification_report(dev.label, LRT))


In [105]:
# test on same train set which is full train set
auc_train

[1.0, 1.0, 1.0, 1.0, 0.9999999999999999]

In [106]:
# test on same dev set which is full dev set
auc_dev

[0.8615730231547929,
 0.8660643505025334,
 0.8651347932387653,
 0.8584328078573867,
 0.852945884097871]

In [22]:
probs = LR.predict_proba(dev.iloc[:,4:-1])

In [23]:
metrics.roc_auc_score(dev.label, probs[:, 1])

0.8607886933151749

In [24]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:]))

Counter({0: 1134, 1: 866})

In [25]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

array([5.43407076e-03, 1.20590556e-08, 1.50067007e-04, ...,
       5.45441957e-03, 9.99999646e-01, 9.01589244e-07])

In [26]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logReg_7feat_hypertuned.csv', header = True, index= False)

### Handmade gridsearch

In [13]:
import itertools as it
from sklearn.metrics import accuracy_score
LR = LogisticRegression
LR_param = {'penalty':['l1', 'l2', 'elasticnet'],
            'C':[1.0,0.5,0.1,0.01], 'max_iter':[200],
           'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
models = [('Logistic Regression',LR,LR_param)]

result = []
best = {}
for name, m, params in models:
    keys, values = zip(*params.items())
    permutations = [dict(zip(keys, v)) for v in it.product(*values)]
    
    for p in permutations:
        try:
            mod = m(**p)
            mod.fit(x_train, y_train);
            y_pred = mod.predict(x_dev)
            Acc_scored = accuracy_score(y_dev, y_pred)
            cnt = collections.Counter(mod.predict(test.iloc[:,4:]))
            best[str(p)] = (Acc_scored, cnt)
            #print('Model: ',name, ' HyperParams: ',p,' Accuracy: ',Acc_scored)
        except:
            pass#print(p)

    #best_params = max(best, key = lambda k:best[k][0])
    #result.append((name, best_params, best[best_params][0], best[best_params][1]))  

C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarn

In [14]:
[print(k,v) for k,v in best.items()]

{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'liblinear'} (0.7889218595450049, Counter({0: 1131, 1: 869}))
{'penalty': 'l1', 'C': 1.0, 'max_iter': 200, 'solver': 'saga'} (0.7008902077151336, Counter({0: 1070, 1: 930}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'liblinear'} (0.7884272997032641, Counter({0: 1147, 1: 853}))
{'penalty': 'l1', 'C': 0.5, 'max_iter': 200, 'solver': 'saga'} (0.7008902077151336, Counter({0: 1070, 1: 930}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'liblinear'} (0.786053412462908, Counter({0: 1152, 1: 848}))
{'penalty': 'l1', 'C': 0.1, 'max_iter': 200, 'solver': 'saga'} (0.7008902077151336, Counter({0: 1070, 1: 930}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'liblinear'} (0.7831849653808111, Counter({0: 1160, 1: 840}))
{'penalty': 'l1', 'C': 0.01, 'max_iter': 200, 'solver': 'saga'} (0.7008902077151336, Counter({0: 1070, 1: 930}))
{'penalty': 'l2', 'C': 1.0, 'max_iter': 200, 'solver': 'newton-cg'} (0.786646884272

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### LGBM

In [27]:
import lightgbm as lgbm

train_data = lgbm.Dataset(x_train, pd.Series(y_train))
test_data = lgbm.Dataset(x_dev, pd.Series(y_dev))

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=40)

ModuleNotFoundError: No module named 'lightgbm'

In [326]:
predict

array([0.49348902, 0.49348902, 0.49348902, ..., 0.49348902, 0.49348902,
       0.49348902])

In [328]:
predict= model.predict(test.iloc[:,4:])
collections.Counter(["pos" if i>0.5 else "neg" for i in predict])

Counter({'neg': 1439, 'pos': 561})

## SVM on full training data

In [112]:
# normalisation of data
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
train_full = scaler.fit_transform(train_full)
test_full = scaler.fit_transform(test_full)

In [113]:
SVM = SVC(kernel='rbf', C=1.0)
clf = CalibratedClassifierCV(SVM) 
clf.fit(train_full, train_full_label)

CalibratedClassifierCV(base_estimator=SVC())

In [114]:
collections.Counter(clf.predict(test_full))

Counter({0.0: 1251, 1.0: 749})

In [115]:
results = clf.predict_proba(test_full)
results[:,1]

array([0.06479071, 0.06399281, 0.11928247, ..., 0.11601861, 0.99999939,
       0.0642454 ])

In [116]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('svm_9feat_hyper_full.csv', header = True, index= False)

### Support Vector Machine (SVM) - development and tuning

In [28]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
x_train = scaler.fit_transform(x_train)
x_dev = scaler.fit_transform(x_dev)
test = test.iloc[:,4:]
test = scaler.fit_transform(test)

In [29]:
x_train

array([[0.1354199 , 0.1771918 , 0.49846154, ..., 0.0619469 , 0.08849559,
        0.001001  ],
       [0.16463012, 0.22761792, 0.54      , ..., 0.0619469 , 0.1061947 ,
        0.001001  ],
       [0.16110755, 0.21662624, 0.756     , ..., 0.0619469 , 0.07079646,
        0.001001  ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.07079646, 0.0619469 ,
        0.003003  ],
       [0.        , 0.        , 0.        , ..., 0.07079646, 0.09734515,
        0.003003  ],
       [0.        , 0.        , 0.        , ..., 0.07079646, 0.0619469 ,
        0.003003  ]])

In [30]:
C = [0.001, 0.01, 0.1, 0.5, 0.8, 1.0]
kernels = ['linear', 'poly', 'rbf', 'sigmoid'] # cannot use precomputed as square matrix as input required
for kernel in kernels:
    for c in C:
        SVM = SVC(kernel=kernel, C=c)
        #SVM_predict = SVM.predict(x_test)
        clf = CalibratedClassifierCV(SVM) 
        clf.fit(x_train, y_train)
        clf_predict = clf.predict(x_dev)

        probs = clf.predict_proba(x_dev)
        print("kernel:", kernel, 'c:', c)
        print(metrics.roc_auc_score(y_dev, probs[:, 1]))
        print(metrics.classification_report(y_dev, clf_predict))

kernel: linear c: 0.001
0.804210303868133
              precision    recall  f1-score   support

           0       0.71      0.87      0.78      5055
           1       0.84      0.64      0.73      5055

    accuracy                           0.76     10110
   macro avg       0.77      0.76      0.76     10110
weighted avg       0.77      0.76      0.76     10110

kernel: linear c: 0.01
0.8137276897744984
              precision    recall  f1-score   support

           0       0.71      0.91      0.80      5055
           1       0.87      0.64      0.74      5055

    accuracy                           0.77     10110
   macro avg       0.79      0.77      0.77     10110
weighted avg       0.79      0.77      0.77     10110

kernel: linear c: 0.1
0.81821279476696
              precision    recall  f1-score   support

           0       0.70      0.99      0.82      5055
           1       0.98      0.58      0.73      5055

    accuracy                           0.79     10110
   ma

kernel: sigmoid c: 1.0
0.7825145946517096
              precision    recall  f1-score   support

           0       0.73      0.73      0.73      5055
           1       0.73      0.73      0.73      5055

    accuracy                           0.73     10110
   macro avg       0.73      0.73      0.73     10110
weighted avg       0.73      0.73      0.73     10110



In [22]:
SVM = SVC(kernel='rbf', C=1.0)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_dev)

probs = clf.predict_proba(x_dev)
#print("kernel:", kernel, 'c:', c)
print(metrics.roc_auc_score(y_dev, probs[:, 1]))
print(metrics.classification_report(y_dev, clf_predict))

0.7234561074471613
              precision    recall  f1-score   support

           0       0.63      0.78      0.70      5055
           1       0.71      0.55      0.62      5055

    accuracy                           0.66     10110
   macro avg       0.67      0.66      0.66     10110
weighted avg       0.67      0.66      0.66     10110



In [17]:
probs = clf.predict_proba(x_dev)

In [18]:
probs

array([[2.34791090e-06, 9.99997652e-01],
       [3.46543691e-07, 9.99999653e-01],
       [4.10257686e-09, 9.99999996e-01],
       ...,
       [8.67132316e-01, 1.32867684e-01],
       [8.66075798e-01, 1.33924202e-01],
       [8.65616779e-01, 1.34383221e-01]])

In [32]:
metrics.roc_auc_score(y_dev, probs[:, 1])

0.90392393855522

In [33]:
collections.Counter(clf.predict(test))

Counter({0: 1385, 1: 615})

In [34]:
collections.Counter(y_test)

NameError: name 'y_test' is not defined

In [35]:
results = clf.predict_proba(test)
results[:,1]

array([0.06129185, 0.05327719, 0.19009618, ..., 0.06437202, 0.99999999,
       0.05693839])

In [36]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('svm_7feat_hypertuned.csv', header = True, index= False)

## RF on full training

In [99]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=8, random_state=0, oob_score=True, bootstrap=True).fit(train_full, train_full_label) # max_depth 6 is optimal depth.

In [100]:
collections.Counter(rfc.predict(test_full))

Counter({0.0: 1215, 1.0: 785})

In [101]:
results = rfc.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.00122951, 0.00122951, 0.00322951, ..., 0.98322951, 0.71      ,
       0.00122951])

In [98]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('randomForest_7feat_full.csv', header = True, index= False)

## Random Forest Classifier - dev and tuning

In [27]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=8, random_state=0, oob_score=True, bootstrap=True).fit(x_train, y_train) # max_depth 6 is optimal depth.
rfc_pred = rfc.predict(x_train)

#print(metrics.classification_report(x_train, rfc_pred))

In [42]:
print('Score: ', rfc.score(x_train, y_train))

Score:  1.0


In [43]:
print(rfc.oob_score_)

1.0


In [28]:
probs = rfc.predict_proba(x_train)

In [29]:
probs

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       ...,
       [0.99679233, 0.00320767],
       [0.9985441 , 0.0014559 ],
       [0.99679233, 0.00320767]])

In [31]:
metrics.roc_auc_score(y_train, probs[:, 1])

1.0

In [62]:
collections.Counter(rfc.predict(test.iloc[:,4:]))

Counter({0: 1065, 1: 935})

In [173]:
results = rfc.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.10305407, 0.06810847, 0.63921827, ..., 0.10767577, 1.        ,
       0.29842463])

In [174]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('randomForest_6feat.csv', header = True, index= False)